In [ ]:
!pip install lottery-ticket-pruner
!pip install matplotlib


In [3]:
import os
import shutil
import tensorflow as tf
from tensorflow import keras
from lottery_ticket_pruner import LotteryTicketPruner
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# The Speck cipher and data generation algorithms #
Taken from https://github.com/agohr/deep_speck/blob/master/speck.py

In [4]:
import numpy as np
from os import urandom

def WORD_SIZE():
    return(16);

def ALPHA():
    return(7);

def BETA():
    return(2);

MASK_VAL = 2 ** WORD_SIZE() - 1;

def shuffle_together(l):
    state = np.random.get_state();
    for x in l:
        np.random.set_state(state);
        np.random.shuffle(x);

def rol(x,k):
    return(((x << k) & MASK_VAL) | (x >> (WORD_SIZE() - k)));

def ror(x,k):
    return((x >> k) | ((x << (WORD_SIZE() - k)) & MASK_VAL));

def enc_one_round(p, k):
    c0, c1 = p[0], p[1];
    c0 = ror(c0, ALPHA());
    c0 = (c0 + c1) & MASK_VAL;
    c0 = c0 ^ k;
    c1 = rol(c1, BETA());
    c1 = c1 ^ c0;
    return(c0,c1);

def dec_one_round(c,k):
    c0, c1 = c[0], c[1];
    c1 = c1 ^ c0;
    c1 = ror(c1, BETA());
    c0 = c0 ^ k;
    c0 = (c0 - c1) & MASK_VAL;
    c0 = rol(c0, ALPHA());
    return(c0, c1);

def expand_key(k, t):
    ks = [0 for i in range(t)];
    ks[0] = k[len(k)-1];
    l = list(reversed(k[:len(k)-1]));
    for i in range(t-1):
        l[i%3], ks[i+1] = enc_one_round((l[i%3], ks[i]), i);
    return(ks);

def encrypt(p, ks):
    x, y = p[0], p[1];
    for k in ks:
        x,y = enc_one_round((x,y), k);
    return(x, y);

def decrypt(c, ks):
    x, y = c[0], c[1];
    for k in reversed(ks):
        x, y = dec_one_round((x,y), k);
    return(x,y);

def check_testvector():
  key = (0x1918,0x1110,0x0908,0x0100)
  pt = (0x6574, 0x694c)
  ks = expand_key(key, 22)
  ct = encrypt(pt, ks)
  if (ct == (0xa868, 0x42f2)):
    print("Testvector verified.")
    return(True);
  else:
    print("Testvector not verified.")
    return(False);

#convert_to_binary takes as input an array of ciphertext pairs
#where the first row of the array contains the lefthand side of the ciphertexts,
#the second row contains the righthand side of the ciphertexts,
#the third row contains the lefthand side of the second ciphertexts,
#and so on
#it returns an array of bit vectors containing the same data
def convert_to_binary(arr):
  X = np.zeros((4 * WORD_SIZE(),len(arr[0])),dtype=np.uint8);
  for i in range(4 * WORD_SIZE()):
    index = i // WORD_SIZE();
    offset = WORD_SIZE() - (i % WORD_SIZE()) - 1;
    X[i] = (arr[index] >> offset) & 1;
  X = X.transpose();
  return(X);

#takes a text file that contains encrypted block0, block1, true diff prob, real or random
#data samples are line separated, the above items whitespace-separated
#returns train data, ground truth, optimal ddt prediction
def readcsv(datei):
    data = np.genfromtxt(datei, delimiter=' ', converters={x: lambda s: int(s,16) for x in range(2)});
    X0 = [data[i][0] for i in range(len(data))];
    X1 = [data[i][1] for i in range(len(data))];
    Y = [data[i][3] for i in range(len(data))];
    Z = [data[i][2] for i in range(len(data))];
    ct0a = [X0[i] >> 16 for i in range(len(data))];
    ct1a = [X0[i] & MASK_VAL for i in range(len(data))];
    ct0b = [X1[i] >> 16 for i in range(len(data))];
    ct1b = [X1[i] & MASK_VAL for i in range(len(data))];
    ct0a = np.array(ct0a, dtype=np.uint16); ct1a = np.array(ct1a,dtype=np.uint16);
    ct0b = np.array(ct0b, dtype=np.uint16); ct1b = np.array(ct1b, dtype=np.uint16);
    
    #X = [[X0[i] >> 16, X0[i] & 0xffff, X1[i] >> 16, X1[i] & 0xffff] for i in range(len(data))];
    X = convert_to_binary([ct0a, ct1a, ct0b, ct1b]); 
    Y = np.array(Y, dtype=np.uint8); Z = np.array(Z);
    return(X,Y,Z);

#baseline training data generator
def make_train_data(n, nr, diff=(0x0040,0)):
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);
  plain1l[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  plain1r[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);

#real differences data generator
def real_differences_data(n, nr, diff=(0x0040,0)):
  #generate labels
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  #generate keys
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);
  #generate plaintexts
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);
  #apply input difference
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);
  #expand keys and encrypt
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  #generate blinding values
  k0 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  k1 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  #apply blinding to the samples labelled as random
  ctdata0l[Y==0] = ctdata0l[Y==0] ^ k0; ctdata0r[Y==0] = ctdata0r[Y==0] ^ k1;
  ctdata1l[Y==0] = ctdata1l[Y==0] ^ k0; ctdata1r[Y==0] = ctdata1r[Y==0] ^ k1;
  #convert to input data for neural networks
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);

# Create folders for saving models for later use if needed #

In [5]:
wdir_curent_bw = "./current_bw/";
wdir_curent_bw_all= "./current_bw_all/";
init_trained= "./trained/";
init_trained_all= "./trained_all/";

os.mkdir(wdir_curent_bw);
os.mkdir(wdir_curent_bw_all);
os.mkdir(init_trained);
os.mkdir(init_trained_all);

os.mkdir("./best");
os.mkdir("./best_all");
os.mkdir("./worst");
os.mkdir("./worst_all");

# The depth-1/10 distinguisher implementation#
Taken from https://github.com/agohr/deep_speck/blob/master/train_nets.py and slightly adapted for running multiple trials. 

In [6]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Conv1D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, MaxPooling1D, Concatenate,Dropout, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.regularizers import l2, l1, l1_l2


def cyclic_lr(num_epochs, high_lr, low_lr):
  res = lambda i: low_lr + ((num_epochs-1) - i % num_epochs)/(num_epochs-1) * (high_lr - low_lr);
  return(res);

def make_checkpoint_best_worst(datei):
  res = ModelCheckpoint(datei, monitor='val_acc', save_best_only = True, save_weights_only=True );
  return(res);

def make_checkpoint_all(datei):
  res = ModelCheckpoint(datei, monitor='val_acc', save_best_only = False, save_weights_only=True);
  return(res);


wdir_curent_bw = "./current_bw/";
wdir_curent_bw_all= "./current_bw_all/";

bs = 5000;

def make_resnet(num_blocks=2, num_filters=32, num_outputs=1, d1=64, d2=64, word_size=16, ks=3,depth=5, reg_param=0.0001, final_activation='sigmoid'):
  #Input and preprocessing layers
  inp = Input(shape=(num_blocks * word_size * 2,));
  rs = Reshape((2 * num_blocks, word_size))(inp);
  perm = Permute((2,1))(rs);
  #add a single residual layer that will expand the data to num_filters channels
  #this is a bit-sliced layer
  conv0 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(perm);
  conv0 = BatchNormalization()(conv0);
  conv0 = Activation('relu')(conv0);
  #add residual blocks
  shortcut = conv0;
  for i in range(depth):
    conv1 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut);
    conv1 = BatchNormalization()(conv1);
    conv1 = Activation('relu')(conv1);
    conv2 = Conv1D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1);
    conv2 = BatchNormalization()(conv2);
    conv2 = Activation('relu')(conv2);
    shortcut = Add()([shortcut, conv2]);
  #add prediction head
  flat1 = Flatten()(shortcut);
  dense1 = Dense(d1,kernel_regularizer=l2(reg_param))(flat1);
  dense1 = BatchNormalization()(dense1);
  dense1 = Activation('relu')(dense1);
  dense2 = Dense(d2, kernel_regularizer=l2(reg_param))(dense1);
  dense2 = BatchNormalization()(dense2);
  dense2 = Activation('relu')(dense2);
  out = Dense(num_outputs, activation=final_activation, kernel_regularizer=l2(reg_param))(dense2);
  model = Model(inputs=inp, outputs=out);

  return model

def model_builder(depth):

  model = make_resnet(depth=depth);
  model.compile(
          optimizer='adam',
          loss='binary_crossentropy',
          metrics=['acc']);
  return model;
  



def train_speck_distinguisher(model, num_epochs, num_rounds,rep_trial,wdirbw, wdirall, pruning_method,pruning_trial, X_train, Y_train, X_eval, Y_eval):
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, restore_best_weights= True);
    check_bw = make_checkpoint_best_worst(wdirbw +str(rep_trial)+ pruning_method +str(pruning_trial)+"{epoch:02d}.hdf5");
    check_bw_all = make_checkpoint_all(wdirall+str(rep_trial) + pruning_method +str(pruning_trial)+"{epoch:02d}.hdf5");
    lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));
    history = model.fit(X_train, Y_train, batch_size= bs, epochs=num_epochs,validation_data=(X_eval, Y_eval), callbacks=[lr, stop_early, check_bw, check_bw_all])
    best_epoch = np.argmax(history.history['val_acc']) + 1
    return(model,best_epoch);


Rename folders that keep the best/worst models

In [7]:
def rename_best():
  shutil.rmtree("./best");
  shutil.rmtree("./best_all");

  os.rename(wdir_curent_bw, "./best");
  os.rename(wdir_curent_bw_all, "./best_all");
    
  os.mkdir(wdir_curent_bw);
  os.mkdir(wdir_curent_bw_all);



In [8]:
def rename_worst():
  shutil.rmtree("./worst");
  shutil.rmtree("./worst_all");

  os.rename(wdir_curent_bw, "./worst");
  os.rename(wdir_curent_bw_all, "./worst_all");
    
  os.mkdir(wdir_curent_bw);
  os.mkdir(wdir_curent_bw_all);


For saving the model

In [9]:
def save_model(model, name):
  model_save_name = name;
  #path = F"/content/gdrive/My Drive/{model_save_name}" 
  model.save(name)
  print("\n");

# Evaluate the results #
Taken from https://github.com/agohr/deep_speck/blob/master/eval.py and slightly adapted.

In [10]:
def evaluate(net,X,Y):
    Z = net.predict(X,batch_size=10000).flatten();
    Zbin = (Z > 0.5);
    diff = Y - Z; 
    mse = np.mean(diff*diff);
    n = len(Z); n0 = np.sum(Y==0); n1 = np.sum(Y==1);
    acc = np.sum(Zbin == Y) / n;
    tpr = np.sum(Zbin[Y==1]) / n1;
    tnr = np.sum(Zbin[Y==0] == 0) / n0;
    mreal = np.median(Z[Y==1]);
    high_random = np.sum(Z[Y==0] > mreal) / n0;

    return(acc, tpr, tnr); # added
    #print("Accuracy: ", acc, "TPR: ", tpr, "TNR: ", tnr, "MSE:", mse);
    #print("Percentage of random pairs with score higher than median of real pairs:", 100*high_random);

# Conducting multiple evaluations #


In [11]:
def multiple_evaluations(model, repetitions, num_rounds):
 
  accs = [];
  tprs = [];
  tnrs = [];
  for i in range(0, repetitions):
    X_eval, Y_eval = make_train_data(10**6, num_rounds);

    (acc, tpr, tnr) = evaluate(model, X_eval, Y_eval);
    accs.append(acc);
    tprs.append(tpr);
    tnrs.append(tnr);

  print("Acc: " + str(np.mean(accs)) + str(" +- ") + str(np.std(accs)) + str("\t") + 
        "Tpr:" + str(np.mean(tprs)) + str(" +- ") + str(np.std(tprs)) + str("\t") +
        "Tnr:" + str(np.mean(tnrs)) + str(" +- ") + str(np.std(tnrs)) + str("\t"));
        
  return(np.mean(accs), np.mean(tprs), np.mean(tnrs));




# Creating a plot with values for accuracy/TPR/TNR for the baseline, pruned with one-shot, and pruned with iterative model #

In [12]:
def create_figure(title, x_label, y_label, x0, y0,xy0err, x1, y1, xy1err, x2, y2, xy2err):
    
    fig, ax =  plt.subplots(figsize=(10,10));
    #trained baseline model 
    err= ax.errorbar(x0,y0,color = 'k', yerr=xy0err, ecolor='k', elinewidth=1, capsize=3);
    err[-1][0].set_linestyle('--'); 
    #one shot from initial
    err=ax.errorbar(x1,y1, color ='y',yerr=xy1err, ecolor="y", elinewidth=1, capsize=3);
    err[-1][0].set_linestyle('--');
    #iterative from initial
    err=ax.errorbar(x2,y2, color='g', yerr=xy2err, ecolor="g", elinewidth=1, capsize=3);
    err[-1][0].set_linestyle('--');
  
    plt.xlabel(x_label);
    plt.ylabel(y_label);

    plt.savefig(title +".png", bbox_inches='tight')

# Pruning a model using the one shot or iterative pruning method for a value of *p%* and training it #

In [13]:
def apply_pruning_method(max_acc, min_acc,pruning_method, num_repetition, depth, num_rounds, X_train, Y_train, X_eval, Y_eval, num_epochs, num_pruning_trials, initial_baseline_model, initial_baseline_model_weights, trained_baseline_model, trained_baseline_model_weights, prune_strategy, prune_percentage_for_iterative):
    
    pruner = LotteryTicketPruner(initial_baseline_model);

    pruned_accs =[];
    pruned_tprs = [];
    pruned_tnrs = [];
    pruning_trial_no=0;
    


    for prune_percentage in list(np.linspace(start=0.1,stop=1,num=num_pruning_trials+1))[0:-1]:
        pruning_trial_no= pruning_trial_no+1;
        if pruning_method=="one_shot":
          pruner.reset_masks();
        elif pruning_method=="iterative":
          prune_percentage = prune_percentage_for_iterative;
          

        initial_baseline_model.set_weights(initial_baseline_model_weights);
        trained_baseline_model.set_weights(trained_baseline_model_weights);

        pruner.calc_prune_mask(trained_baseline_model, prune_percentage, prune_strategy);

      
        pruner.apply_pruning(initial_baseline_model);



        (pruned_trained_model, stoped_epoch) = train_speck_distinguisher(initial_baseline_model,num_epochs, num_rounds, num_repetition, wdir_curent_bw, wdir_curent_bw_all,pruning_method,pruning_trial_no, X_train,  Y_train, X_eval, Y_eval);

        print("Experiment repetition no:"+ str(num_repetition)+ " Pruned model at " + str(prune_percentage) + "pruning trial:"+ str(pruning_trial_no)+"\n");
        (acc, tpr, tnr) = multiple_evaluations(pruned_trained_model,10,num_rounds);

        if(acc >=max_acc):
          max_acc = acc;
          rename_best();
        if(acc <=min_acc):
          min_acc = acc;
          rename_worst();

        pruned_accs.append(acc);
        pruned_tprs.append(tpr);
        pruned_tnrs.append(tnr);

        


    return(max_acc,min_acc,pruned_accs, pruned_tprs, pruned_tnrs)


# Execute the above described pruning for different values of *p%* multiple times#

In [14]:
def execute_experiments(num_rounds=5, num_epochs=40, num_experiment_repetitions=5, num_pruning_trials= 9, depth=10,data_for_experiment="Real_vs_Random",prune_strategy="smallest_weights_global", prune_percentage_for_iterative=0.2):
    
    max_acc =0;
    min_acc =1;
    
    baseline_accuracies_all_trials=[];
    baseline_tprs_all_trials=[];
    baseline_tnrs_all_trials=[];

    pruned_accuracies_all_trials_one_shot_from_initial=[[] for x in range(num_pruning_trials)];
    pruned_tprs_all_trials_one_shot_from_initial=[[] for x in range(num_pruning_trials)];
    pruned_tnrs_all_trials_one_shot_from_initial=[[] for x in range(num_pruning_trials)];

    pruned_accuracies_all_trials_iterative_from_initial=[[] for x in range(num_pruning_trials)];
    pruned_tprs_all_trials_iterative_from_initial=[[] for x in range(num_pruning_trials)];
    pruned_tnrs_all_trials_iterative_from_initial=[[] for x in range(num_pruning_trials)];

   
    
    for repetition in range(0,num_experiment_repetitions):

        if data_for_experiment=="Real_vs_Random":      
            X_train, Y_train = make_train_data(10**7,num_rounds);
            X_eval, Y_eval = make_train_data(10**6, num_rounds);
        elif data_for_experiment=="Real_differences":
            X_train, Y_train = make_train_data(10**7,num_rounds);
            X_eval, Y_eval = make_train_data(10**6, num_rounds);

        initial_baseline_model = model_builder(depth);
        initial_baseline_model_weights = initial_baseline_model.get_weights();
        save_model(initial_baseline_model,"Initial baseline model "+str(num_rounds) +"R" + " Experiment trial no:"+str(repetition));


        (trained_baseline_model,trnd_bm_stop_epoch) = train_speck_distinguisher(initial_baseline_model,num_epochs, num_rounds, repetition,init_trained, init_trained_all,"init_trained",repetition, X_train, Y_train, X_eval, Y_eval);
        trained_baseline_model_weights = trained_baseline_model.get_weights();
        save_model(trained_baseline_model,"Trained baseline model "+str(num_rounds) +"R" + "Experiment trial no:"+str(repetition));

        print("Baseline model results at repetition "+str(repetition)+":\n");
        (b_acc, b_tpr, b_tnr) =multiple_evaluations(trained_baseline_model,10,num_rounds);
        baseline_accuracies_all_trials.append(b_acc);
        baseline_tprs_all_trials.append(b_tpr);
        baseline_tnrs_all_trials.append(b_tnr);

        print("Starting One-Shot with initialization from original initial network:\n");
        (max_acc,min_acc,oneShot_from_initial_pruned_accs, oneShot_from_initial_pruned_tprs, oneShot_from_initial_pruned_tnrs)=apply_pruning_method(max_acc,min_acc,"one_shot", repetition,depth, num_rounds, X_train, Y_train, X_eval, Y_eval, trnd_bm_stop_epoch, num_pruning_trials, initial_baseline_model, initial_baseline_model_weights, trained_baseline_model, trained_baseline_model_weights, prune_strategy, prune_percentage_for_iterative);
        print("Starting Iterative with initialization from original initial network:\n");
        (max_acc, min_acc,iterative_from_initial_pruned_accs, iterative_from_initial_pruned_tprs, iterative_from_initial_pruned_tnrs)=apply_pruning_method(max_acc,min_acc,"iterative", repetition, depth, num_rounds, X_train, Y_train, X_eval, Y_eval, trnd_bm_stop_epoch, num_pruning_trials, initial_baseline_model, initial_baseline_model_weights, trained_baseline_model, trained_baseline_model_weights, prune_strategy, prune_percentage_for_iterative);
  
        for i in range(0,num_pruning_trials):
            pruned_accuracies_all_trials_one_shot_from_initial[i].append(oneShot_from_initial_pruned_accs[i]);
            pruned_tprs_all_trials_one_shot_from_initial[i].append(oneShot_from_initial_pruned_tprs[i]);
            pruned_tnrs_all_trials_one_shot_from_initial[i].append(oneShot_from_initial_pruned_tnrs[i]);

            pruned_accuracies_all_trials_iterative_from_initial[i].append(iterative_from_initial_pruned_accs[i]);
            pruned_tprs_all_trials_iterative_from_initial[i].append(iterative_from_initial_pruned_tprs[i]);
            pruned_tnrs_all_trials_iterative_from_initial[i].append(iterative_from_initial_pruned_tnrs[i]);

        
    return (    baseline_accuracies_all_trials, baseline_tprs_all_trials, baseline_tnrs_all_trials,
                pruned_accuracies_all_trials_one_shot_from_initial, pruned_tprs_all_trials_one_shot_from_initial, pruned_tnrs_all_trials_one_shot_from_initial, 
                pruned_accuracies_all_trials_iterative_from_initial, pruned_tprs_all_trials_iterative_from_initial, pruned_tnrs_all_trials_iterative_from_initial,
           );
            

Compute *p* values for iterative pruning

In [15]:
def compute_pruning_percentages_iterative(num_pruning_trials, pruning_percentage):
  remaining_weights=1;
  percentages=[];

  for i in range(0, num_pruning_trials):
    remaining_weights= remaining_weights - remaining_weights*pruning_percentage;
    percentages.append(1- remaining_weights);


  return (percentages);

  

# Create the Accuracy, TPR and TNR plots #

In [16]:
def create_plots_for_executed_experiments(experiment_type, depth, num_rounds, num_experiment_repetitions, num_pruning_trials, prune_percentage_for_iterative, b_accs, b_tprs, b_tnrs, os_init_accs, os_init_tprs, os_init_tnrs, it_init_accs, it_init_tprs, it_init_tnrs):

      print("Experiment type: "+experiment_type);
      print("Model depth: "+ str(depth));
      print("Speck reduced to "+str(num_rounds) +" R");

      pruning_percentages_one_shot = list(np.linspace(start=0.1,stop=1,num=num_pruning_trials+1))[0:-1];
      pruning_percentages_iterative = compute_pruning_percentages_iterative(num_pruning_trials, prune_percentage_for_iterative);

      b_accs0 =[];
      b_accs0err=[[],[]];
      b_tprs0 =[];
      b_tprs0err = [[],[]];
      b_tnrs0 = [];
      b_tnrs0err = [[],[]]; 
        
        
      b_accs0 = [np.mean(b_accs)] *  num_pruning_trials;
      b_accs0err[0] = [np.mean(b_accs)-np.min(b_accs)] * num_pruning_trials;
      b_accs0err[1] = [np.max(b_accs) -np.mean(b_accs)] * num_pruning_trials;

      b_tprs0 = [np.mean(b_tprs)] * num_pruning_trials;
      b_tprs0err[0] = [np.mean(b_tprs)-np.min(b_tprs)]* num_pruning_trials;
      b_tprs0err[1] = [np.max(b_tprs) -np.mean(b_tprs)]* num_pruning_trials;

      b_tnrs0 = [np.mean(b_tnrs)] * num_pruning_trials;
      b_tnrs0err[0] = [np.mean(b_tnrs)-np.min(b_tnrs)] * num_pruning_trials;
      b_tnrs0err[1] = [np.max(b_tnrs)-np.mean(b_tnrs)] * num_pruning_trials;
      

      os_init_accs1= [];
      os_init_accs1err= [[],[]];

      os_init_tprs1= [];
      os_init_tprs1err= [[],[]];

      os_init_tnrs1= [];
      os_init_tnrs1err= [[],[]];
      

      it_init_accs3= [];
      it_init_accs3err= [[],[]];

      it_init_tprs3= [];
      it_init_tprs3err= [[],[]];

      it_init_tnrs3= [];
      it_init_tnrs3err= [[],[]];
      



      for repetition in range(0, num_pruning_trials):
        
          os_init_accs1.append(np.mean(os_init_accs[repetition]));
          os_init_accs1err[0].append(np.mean(os_init_accs[repetition])- np.min(os_init_accs[repetition]));
          os_init_accs1err[1].append(np.max(os_init_accs[repetition]) - np.mean(os_init_accs[repetition]));

          os_init_tprs1.append(np.mean(os_init_tprs[repetition]));
          os_init_tprs1err[0].append(np.mean(os_init_tprs[repetition]) - np.min(os_init_tprs[repetition]));
          os_init_tprs1err[1].append(np.max(os_init_tprs[repetition]) - np.mean(os_init_tprs[repetition]));

          os_init_tnrs1.append(np.mean(os_init_tnrs[repetition]));
          os_init_tnrs1err[0].append(np.mean(os_init_tnrs[repetition])- np.min(os_init_tnrs[repetition]));
          os_init_tnrs1err[1].append(np.max(os_init_tnrs[repetition]) -np.mean(os_init_tnrs[repetition]));
        

          it_init_accs3.append(np.mean(it_init_accs[repetition]));
          it_init_accs3err[0].append(np.mean(it_init_accs[repetition]) - np.min(it_init_accs[repetition]));
          it_init_accs3err[1].append(np.max(it_init_accs[repetition]) - np.mean(it_init_accs[repetition]));

          it_init_tprs3.append(np.mean(it_init_accs[repetition]));
          it_init_tprs3err[0].append(np.mean(it_init_accs[repetition])-np.min(it_init_accs[repetition]));
          it_init_tprs3err[1].append(np.max(it_init_accs[repetition]) - np.mean(it_init_accs[repetition]));

          it_init_tnrs3.append(np.mean(it_init_tnrs[repetition]));
          it_init_tnrs3err[0].append(np.mean(it_init_tnrs[repetition])-np.min(it_init_tnrs[repetition]));
          it_init_tnrs3err[1].append(np.max(it_init_tnrs[repetition]) - np.mean(it_init_tnrs[repetition]));


   
      create_figure("Accuracies-"+"R:"+str(num_rounds)+"-"+experiment_type,"Pruned ratio", "Accuracy", pruning_percentages_one_shot, b_accs0, b_accs0err, pruning_percentages_one_shot, os_init_accs1, os_init_accs1err, pruning_percentages_iterative, it_init_accs3, it_init_accs3err);
      create_figure("TPRS-"+"R:"+str(num_rounds)+"-"+experiment_type,"Pruned ratio", "TPR", pruning_percentages_one_shot, b_tprs0, b_tprs0err, pruning_percentages_one_shot, os_init_tprs1, os_init_tprs1err, pruning_percentages_iterative, it_init_tprs3, it_init_tprs3err);
      create_figure("TNRS-"+"R:"+str(num_rounds) +"-"+experiment_type,"Pruned ratio", "TNR", pruning_percentages_one_shot, b_tnrs0, b_tnrs0err, pruning_percentages_one_shot, os_init_tnrs1, os_init_tnrs1err, pruning_percentages_iterative, it_init_tnrs3, it_init_tnrs3err);
    





# Running the experiment #

In [17]:
num_rounds=5; # 6, 7, 8
num_epochs=20;
num_experiment_repetitions=5;
num_pruning_trials= 9;
depth=10; # 1
data_for_experiment="Real_vs_Random";
experiment_type= data_for_experiment;
prune_strategy="smallest_weights_global";
prune_percentage_for_iterative=0.2;
     

In [ ]:
(   b_accs, b_tprs, b_tnrs,
    os_init_accs, os_init_tprs, os_init_tnrs,
    it_init_accs, it_init_tprs, it_init_tnrs) = execute_experiments(num_rounds, num_epochs, num_experiment_repetitions, num_pruning_trials, depth,data_for_experiment,prune_strategy, prune_percentage_for_iterative);
     

In [ ]:
create_plots_for_executed_experiments(experiment_type, depth, num_rounds, num_experiment_repetitions, num_pruning_trials, prune_percentage_for_iterative, b_accs, b_tprs, b_tnrs, os_init_accs, os_init_tprs, os_init_tnrs, it_init_accs, it_init_tprs, it_init_tnrs)
